In [1]:
import pandas as pd
import os.path
from nltk.tokenize import word_tokenize
import nltk

DATAPATH = "data/trial"

class exerpt:
    def __init__(self, name, txt, ann, tsv):
        self.name = name
        self.txt = txt
        self.ann = ann
        self.tsv = tsv
        self.context = False
        self.posTag = False
        
    def getContext(self, count = 3):
        if self.context == False: 
            beforeContext = []
            afterContext = []
            self.context = True
            for start,end in zip(self.tsv["startOffset"].values, self.tsv["endOffset"].values ):#self.tsv.iterrows():
                a = getIndexSeperated(exerpt.txt[0],end,count = count)
                b = getIndexSeperated(exerpt.txt[0],start,count = count,forward = False)
                beforeContext.append(self.txt[0][b:start])
                afterContext.append(self.txt[0][end:a])
            self.tsv.insert(3,"beforeContext", beforeContext)
            self.tsv.insert(3,"afterContext", afterContext)
            
    def getPosTag(self):
        if self.context and self.posTag == False: 
            bTag = []
            aTag = []
            self.posTag = True
            for before, after in zip(self.tsv["beforeContext"].values,self.tsv["afterContext"].values):
                bTag.append(nltk.pos_tag(word_tokenize(before)))
                aTag.append(nltk.pos_tag(word_tokenize(after)))
            self.tsv.insert(3,"beforeTag", bTag)
            self.tsv.insert(3,"afterTag", aTag)
            


def readTXTByLine(filepath):
    ftemp = open(filepath, "r", encoding = "utf-8")
    raw = []
    for line in ftemp: 
        raw.append(line)
    ftemp.close()
    return raw

data = {}

with open("data/trial/prefixes.txt","r") as f:
    for l in f:
        l = l[:-1]
        data[l] = exerpt(
            l,
            readTXTByLine(os.path.join(DATAPATH, "txt", l + ".txt")),
            readTXTByLine(os.path.join(DATAPATH, "ann", l + ".ann")),
            pd.read_csv(os.path.join(DATAPATH, "tsv", l + ".tsv"), "\t", header = 0 )
        )

In [2]:
        
def getIndexSeperated(text, index, count=3, char=" ", forward=True):
    """
    Description: method that searches for the next index(depending on direction), where 
    there is an occurence of exactly count words between the current word
    
    count: number of characters to look for 
    text: the string to look through
    index: the index of the string to start at
    """
    if forward == True: 
        r = range(index, len(text))
    else:
        r = reversed(range(0, index))
    
    charCount = 0
    for i in r:
        if text[i] == " ":
            charCount += 1
        
        if charCount == count + 1:
            return i
        
    if forward == True:
        return len(text) - 1
    else :
        return 0

    

In [3]:
count =1
h = []
for exerpt in data.values():
    exerpt.getContext(7)
    exerpt.getPosTag()
#     for l in exerpt.tsv.iterrows():
#         if count ==1 :
#             print(l[1][4])
#             a = getIndexSeperated(exerpt.txt[0],l[1][4],count = 7)
#             b = getIndexSeperated(exerpt.txt[0],l[1][3],count = 7,forward = False)
#             h.append(exerpt.txt[0][b:a])
    count = 90

In [4]:
for column in data["S0012821X12004384-1302"].tsv["beforeTag"].values:
    print(column[0])

('axes', 'NNS')
('(', '(')
('Five', 'CD')
('(', '(')
('highest', 'JJS')
('3', 'CD')
('(', '(')
('near', 'IN')
('the', 'DT')
('younger', 'JJR')
('the', 'DT')


In [5]:
qCounts = {}
mCounts = {}
qText = []
mText = []
for exerpt in data.values():
    for text, annotType, bTag, aTag in zip(exerpt.tsv["text"].values, exerpt.tsv["annotType"].values,exerpt.tsv["beforeTag"].values,exerpt.tsv["afterTag"].values):
        if(annotType == "Quantity"):
            bCount = 0
            num = 1
            while(bCount < len(bTag)):
                #if bTag[len(bTag) - bCount - 1][0].isalnum():
                try:
                    qCounts["b" + str(num)].append(bTag[len(bTag) - bCount - 1][1])
                    num += 1
                except KeyError:
                    qCounts["b" + str(num)] = [bTag[len(bTag) - bCount - 1][1]]
                    num += 1
                bCount +=1
                
            aCount = 0
            num = 1
            while(aCount < len(aTag)):
                #if aTag[aCount][0].isalnum():
                try:
                    qCounts["a" + str(num)].append(aTag[aCount][1])
                    num += 1
                except KeyError:
                    qCounts["a" + str(num)] = [aTag[aCount][1]]
                    num += 1
                aCount +=1
                
            qText.append(text)
                    
        elif(annotType == "MeasuredEntity"):
            bCount = 0
            num = 1
            while(bCount < len(bTag)):
                #if bTag[len(bTag) - bCount - 1][0].isalnum():
                try:
                    mCounts["b" + str(num)].append(bTag[len(bTag) - bCount - 1][1])
                    num += 1
                except KeyError:
                    mCounts["b" + str(num)] = [bTag[len(bTag) - bCount - 1][1]]
                    num += 1
                bCount +=1
                
            aCount = 0
            num = 1
            while(aCount < len(aTag)):
                #if aTag[aCount][0].isalnum():
                try:
                    mCounts["a" + str(num)].append(aTag[aCount][1])
                    num += 1
                except KeyError:
                    mCounts["a" + str(num)] = [aTag[aCount][1]]
                    num += 1
                aCount +=1
            
            mText.append(text)
        
            
    

In [6]:
qCounts

{'b1': ['.',
  'IN',
  '(',
  'DT',
  'NNS',
  'IN',
  'IN',
  ',',
  'VBZ',
  'NN',
  'VBZ',
  'JJ',
  '(',
  'NN',
  'NN',
  'RB',
  'DT',
  'VBD',
  'DT',
  'CC',
  'IN',
  'IN',
  'IN',
  'IN',
  'TO',
  'TO',
  'IN',
  'IN',
  'CC',
  'VBD',
  'IN',
  'NN',
  '(',
  'IN',
  '(',
  'IN',
  'TO',
  'DT',
  'NNS',
  '(',
  'IN',
  'DT',
  'CC',
  'NN',
  'DT',
  '(',
  '(',
  'IN',
  'IN',
  'IN',
  'DT',
  '(',
  'DT',
  'DT',
  'NN',
  'IN',
  ',',
  'TO',
  'NNP',
  'NN',
  'DT',
  '(',
  'VBD',
  'IN',
  'IN',
  '(',
  'IN',
  'IN',
  'VB',
  '(',
  '(',
  'VBN',
  'VBG',
  'IN',
  'NNS',
  'DT',
  '(',
  'IN',
  'DT',
  'IN',
  'NNP',
  'NNP',
  'CC',
  'JJ',
  'IN',
  'IN',
  'NNS',
  'IN',
  'IN',
  'IN',
  'IN',
  'IN',
  'CC',
  'IN',
  'IN',
  'IN',
  'IN',
  'IN',
  'TO',
  'VBD',
  'NNS',
  'IN',
  'TO',
  'TO',
  'NNS',
  ',',
  'CC',
  'IN',
  'IN',
  'IN',
  'IN',
  'IN',
  'IN',
  'IN',
  '(',
  ',',
  'TO',
  'NNS',
  'IN',
  'TO',
  'TO',
  'IN',
  ',',
  'CC',
  'I

In [7]:
for k in qCounts.keys():
    qCounts[k] = nltk.FreqDist(qCounts[k])
    
for k in mCounts.keys():
    mCounts[k] = nltk.FreqDist(mCounts[k])

In [8]:
qCounts

{'b1': FreqDist({'IN': 120, 'DT': 24, '(': 23, 'NN': 23, 'CC': 18, 'TO': 14, 'NNS': 10, ',': 10, 'VBZ': 10, 'NNP': 9, ...}),
 'b2': FreqDist({'NN': 98, 'NNS': 35, 'IN': 28, 'NNP': 24, 'CD': 23, 'VBN': 16, ')': 6, 'RB': 6, 'DT': 6, 'TO': 6, ...}),
 'b3': FreqDist({'NN': 43, 'JJ': 40, 'DT': 33, 'IN': 31, 'NNP': 30, 'CD': 17, 'NNS': 17, 'VBN': 12, 'VBD': 9, '(': 7, ...}),
 'b4': FreqDist({'IN': 60, 'NN': 48, 'DT': 46, 'JJ': 25, 'CD': 21, 'NNP': 16, 'NNS': 12, 'VBD': 11, 'VBZ': 8, '.': 7, ...}),
 'b5': FreqDist({'NN': 54, 'IN': 49, 'DT': 30, 'NNS': 23, 'NNP': 19, 'JJ': 16, ',': 11, 'RB': 10, 'CC': 9, '.': 8, ...}),
 'b6': FreqDist({'NN': 61, 'IN': 37, 'DT': 25, 'NNP': 22, 'JJ': 20, 'CD': 19, 'NNS': 18, ')': 12, '.': 10, ',': 9, ...}),
 'b7': FreqDist({'NN': 51, 'IN': 37, 'DT': 36, 'CD': 29, 'NNP': 24, 'JJ': 20, ',': 13, 'NNS': 12, 'VBN': 9, 'VBZ': 9, ...}),
 'b8': FreqDist({'NN': 25, 'IN': 17, 'CD': 16, 'NNP': 11, 'NNS': 10, ',': 9, 'RB': 9, 'DT': 9, 'JJ': 8, 'VBN': 7, ...}),
 'b9': FreqDi

In [9]:
qText = [nltk.pos_tag(word_tokenize(text)) for text in qText]
mText = [nltk.pos_tag(word_tokenize(text)) for text in mText]

In [10]:
mText

[[('samples', 'NNS')],
 [('samples', 'NNS')],
 [('samples', 'NNS')],
 [('samples', 'NNS')],
 [('samples', 'NNS')],
 [('specimens', 'NNS')],
 [('Samples', 'NNS')],
 [('Dinoflagellate', 'NNP'),
  ('cyst', 'NN'),
  ('assemblage', 'NN'),
  ('1', 'CD'),
  ('(', '('),
  ('DA1', 'NNP')],
 [('DA1', 'NN')],
 [('CIE', 'NNP'), ('onset', 'NN')],
 [('northern', 'JJ'), ('rain', 'NN'), ('belt', 'NN')],
 [('CIE', 'NNP'), ('onset', 'NN')],
 [('CIE', 'NNP'), ('onset', 'NN')],
 [('Oi1', 'NN')],
 [('ice', 'NN'), ('volume', 'NN')],
 [('ice', 'NN'), ('volume', 'NN')],
 [('Oi1', 'NN')],
 [('timescale', 'NN')],
 [('decline', 'NN')],
 [('fraction', 'NN')],
 [('spicules', 'NNS')],
 [('concentrated', 'VBN'), ('HNO3', 'NNP')],
 [('wet', 'JJ'), ('alkaline', 'NN'), ('digestion', 'NN')],
 [('wet', 'JJ'), ('alkaline', 'NN'), ('digestion', 'NN')],
 [('wet', 'JJ'), ('alkaline', 'NN'), ('digestion', 'NN')],
 [('wet', 'JJ'), ('alkaline', 'NN'), ('digestion', 'NN')],
 [('samples', 'NNS')],
 [('thermally', 'RB'), ('distill

In [11]:
txt = {"textQ":[pos[1] for n in qText for pos in n],"textM":[pos[1] for n in mText for pos in n]}

In [12]:
for k in txt.keys():
    txt[k] = nltk.FreqDist(txt[k])


In [13]:
txt

{'textQ': FreqDist({'CD': 302, 'NN': 204, 'NNP': 41, 'IN': 33, 'TO': 20, 'NNS': 20, 'RB': 20, 'CC': 18, 'JJ': 17, ',': 9, ...}),
 'textM': FreqDist({'NN': 250, 'NNS': 78, 'NNP': 76, 'JJ': 76, 'IN': 30, 'DT': 13, 'CD': 9, 'VBN': 6, 'RB': 6, 'VBZ': 5, ...})}

In [14]:
start, end = (data["S0012821X12004384-1302"].tsv["startOffset"].values,data["S0012821X12004384-1302"].tsv["endOffset"].values)

In [15]:
for start, end in zip(data["S0012821X12004384-1302"].tsv["startOffset"].values,data["S0012821X12004384-1302"].tsv["endOffset"].values):
    print(start)

553
558
588
558
571
769
760
1481
1485
1493
1485


In [16]:
data["S0012821X12004384-1302"].tsv

,docId,annotSet,annotType,afterTag,beforeTag,afterContext,beforeContext,startOffset,endOffset,annotId,text,other
0,S0012821X12004384-1302,1,Quantity,"[(samples, NNS), (from, IN), (below, IN), (the...","[(axes, NNS), ((, (), (describing, VBG), (the,...","samples from below the CIE at 2619.60,",axes (describing the highest percentages of v...,553,557,T9,Five,"{mods: [IsCount], unit: count}"
1,S0012821X12004384-1302,1,MeasuredEntity,"[(from, IN), (below, IN), (the, DT), (CIE, NNP...","[((, (), (describing, VBG), (the, DT), (highes...","from below the CIE at 2619.60, 2617.35,",(describing the highest percentages of varian...,558,565,T10,samples,{HasQuantity: T9}
2,S0012821X12004384-1302,2,Quantity,"[((, (), (indicated, VBN), (in, IN), (Fig, NNP...","[(Five, CD), (samples, NNS), (from, IN), (belo...","(indicated in Fig. 5) contain Apectodinium, in",Five samples from below the CIE at,588,637,T1,"2619.60, 2617.35, 2617.44, 2614.73, and 2614.71 m","{mods: [IsList], unit: m}"
3,S0012821X12004384-1302,2,MeasuredEntity,"[(from, IN), (below, IN), (the, DT), (CIE, NNP...","[((, (), (describing, VBG), (the, DT), (highes...","from below the CIE at 2619.60, 2617.35,",(describing the highest percentages of varian...,558,565,T10,samples,{HasQuantity: T1}
4,S0012821X12004384-1302,2,Qualifier,"[(at, IN), (2619.60, CD), (,, ,), (2617.35, CD...","[(highest, JJS), (percentages, NNS), (of, IN),...","at 2619.60, 2617.35, 2617.44, 2614.73, and 26...",highest percentages of variance). Five sample...,571,584,T15,below the CIE,{Qualifies: T1}
5,S0012821X12004384-1302,3,Quantity,"[(), )), (also, RB), (contain, VBP), (negative...","[(3, CD), (and, CC), (7, CD), (), )), (., .), ...",") also contain negative δ13CTOC, indicative of...",3 and 7). Some of these samples (,769,800,T2,"2619.60, 2614.73, and 2614.71 m","{mods: [IsList], unit: m}"
6,S0012821X12004384-1302,3,MeasuredEntity,"[((, (), (2619.60, CD), (,, ,), (2614.73, CD),...","[((, (), (Figs, NNP), (., .), (3, CD), (and, C...","(2619.60, 2614.73, and 2614.71 m) also contain",(Figs. 3 and 7). Some of these,760,767,T11,samples,{HasQuantity: T2}
7,S0012821X12004384-1302,4,Quantity,"[(samples, NNS), (2619.60, CD), (and, CC), (26...","[(near, IN), (the, DT), (younger, JJR), (sampl...",samples 2619.60 and 2614.71 have a spore,near the younger samples from PA4. The,1481,1484,T3,two,"{mods: [IsCount], unit: count}"
8,S0012821X12004384-1302,4,MeasuredEntity,"[(2619.60, CD), (and, CC), (2614.71, CD), (hav...","[(the, DT), (younger, JJR), (samples, NNS), (f...",2619.60 and 2614.71 have a spore and,the younger samples from PA4. The two,1485,1492,T14,samples,{HasQuantity: T3}
9,S0012821X12004384-1302,5,Quantity,"[(have, VB), (a, DT), (spore, NN), (and, CC), ...","[(younger, JJR), (samples, NNS), (from, IN), (...",have a spore and pollen palynological signature,younger samples from PA4. The two samples,1493,1512,T13,2619.60 and 2614.71,"{mods: [IsList], unit: Unspecified}"


In [17]:
data["S0012821X12004384-1302"].txt[0][553:557]

'Five'

In [18]:
data["S0012821X12004384-1302"].txt

['Correspondence analysis (CA) and statistical diversity analysis were carried out on the palynological dataset (total counts per gram) to confirm assemblage designations (Figs. 4 and 5), to identify any disturbance to the core prior to interpretation, and to estimate diversity (Fig. 6). Dinoflagellate cyst assemblages (DA1–DA5) and pollen assemblages (PA1–PA4) were defined by visually comparing changes in the species dominance (Figs. 7 and 8), and confirmed by CA (Fig. 4) using the first three axes (describing the highest percentages of variance). Five samples from below the CIE at 2619.60, 2617.35, 2617.44, 2614.73, and 2614.71 m (indicated in Fig. 5) contain Apectodinium, in contrast to the other samples below the CIE (Figs. 3 and 7). Some of these samples (2619.60, 2614.73, and 2614.71 m) also contain negative δ13CTOC, indicative of the CIE (Fig. 3). To test if coincident pollen and spore changes also occur in these samples, we used CA on the pollen and spore data only (Fig. 5a). P

In [19]:
data["S0012821X12004384-1302"].ann

['T1\tQuantity 588 637\t2619.60, 2617.35, 2617.44, 2614.73, and 2614.71 m\n',
 'T2\tQuantity 769 800\t2619.60, 2614.73, and 2614.71 m\n',
 'T9\tQuantity 553 557\tFive\n',
 'T10\tMeasuredEntity 558 565\tsamples\n',
 'T11\tMeasuredEntity 760 767\tsamples\n',
 'R7\tHasQuantity Arg1:T11 Arg2:T2\t\n',
 'T13\tQuantity 1493 1512\t2619.60 and 2614.71\n',
 'T14\tMeasuredEntity 1485 1492\tsamples\n',
 'R10\tHasQuantity Arg1:T14 Arg2:T13\t\n',
 "#5\tAnnotatorNotes T10\tProperty for the quantities is probably 'depth'\n",
 'A1\tIsCount T9\n',
 'R11\tHasQuantity Arg1:T10 Arg2:T9\t\n',
 'T15\tQualifier 571 584\tbelow the CIE\n',
 'A2\tQuantityQualifier T1 IsList\n',
 '#1\tAnnotatorNotes T1\tm\n',
 'R1\tQualifies Arg1:T15 Arg2:T1\t\n',
 'R2\tHasQuantity Arg1:T10 Arg2:T1\t\n',
 'A3\tQuantityQualifier T2 IsList\n',
 '#2\tAnnotatorNotes T2\tm\n',
 'T3\tQuantity 1481 1484\ttwo\n',
 'A4\tIsCount T3\n',
 'R3\tHasQuantity Arg1:T14 Arg2:T3\t\n',
 'A5\tQuantityQualifier T13 IsList\n',
 '#3\tAnnotatorNotes T13\

In [20]:
import nltk
from nltk.tokenize import word_tokenize
tokens = word_tokenize(data["S0012821X12004384-1302"].txt[0])
#tokens = [tok for tok in tokens if tok.isalnum()]

In [21]:
tagged = nltk.pos_tag(tokens)
len(tagged)

424

In [22]:
def process(words):
    try: 
        chunkgram = r"""measurement : {<NN>*<IN>+<CD|NN|CC>+<IN>+}
                                    {<NN>+<,|(|)>*<CD|NN|CC>+<,|(|)>*<IN>*}
                                    {<NN>+<IN>+<CD|NN|CC>+<IN>+}
                                    {<>+<CD|NN|CC>+<IN>+}
                            """
        chunkparser = nltk.RegexpParser(chunkgram)
        chunked = chunkparser.parse(tagged)
            
        print(chunked)
        print("==============")
        
        chunked.draw()
            
            
    except Exception as e: 
        print(str(e))
        
        
process(tagged)

(S
  (measurement Correspondence/NN analysis/NN)
  (/(
  CA/NNP
  )/)
  and/CC
  statistical/JJ
  (measurement diversity/NN analysis/NN)
  were/VBD
  carried/VBN
  out/RP
  on/IN
  the/DT
  palynological/JJ
  dataset/NN
  (/(
  total/JJ
  counts/NNS
  per/IN
  gram/NN
  )/)
  to/TO
  confirm/VB
  assemblage/NN
  designations/NNS
  (/(
  Figs/NNP
  ./.
  4/CD
  and/CC
  5/CD
  )/)
  ,/,
  to/TO
  identify/VB
  any/DT
  disturbance/NN
  to/TO
  the/DT
  core/NN
  prior/RB
  to/TO
  (measurement interpretation/NN ,/, and/CC)
  to/TO
  estimate/VB
  diversity/NN
  (/(
  Fig/NNP
  ./.
  6/CD
  )/)
  ./.
  Dinoflagellate/NNP
  cyst/NN
  assemblages/NNS
  (/(
  DA1–DA5/NNP
  )/)
  and/CC
  pollen/VBN
  assemblages/NNS
  (/(
  PA1–PA4/NNP
  )/)
  were/VBD
  defined/VBN
  by/IN
  visually/RB
  comparing/VBG
  changes/NNS
  in/IN
  the/DT
  species/NNS
  dominance/NN
  (/(
  Figs/NNP
  ./.
  7/CD
  and/CC
  8/CD
  )/)
  ,/,
  and/CC
  confirmed/VBN
  by/IN
  CA/NNP
  (/(
  Fig/NNP
  ./.
  4/CD
 

In [23]:
"10002".isalnum()


True

In [44]:
server_url = "http://localhost:8060/service"
from grobid_quantities.quantities import QuantitiesClient
client = QuantitiesClient(apiBase=server_url)

In [48]:
responses = {}
for key in data.keys():
    responses[key] = client.process_text(data[key].txt[0])

In [50]:
import json 

for key in responses.keys():
    json.dump(responses[key], open(key+".grobid","w"),indent = 3)